In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from skimage.io import imsave
import os
import shutil
import gzip
img_height = 28
img_width = 28
img_size = img_height * img_width

to_train = True
to_restore = True
output_path = "output2"

max_epoch = 1000

h1_size = 150
h2_size = 300
z_size = 100
batch_size = 256



C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
def load_mnist(path, kind='train'):

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [3]:
# 제너레이터 (G)

    # ----------- 코드 작성부분 ----------------

def generator(z):
    w1 = tf.Variable(tf.truncated_normal([z_size, h1_size],stddev=0.1),name="g_wl", dtype=tf.float32)
    b1 = tf.Variable(tf.zeros([h1_size]),name="g_b1", dtype=tf.float32)
    h1 = tf.nn.relu(tf.matmul(z,w1)+b1)
        
    w2 = tf.Variable(tf.truncated_normal([h1_size, h2_size],stddev=0.1),name="g_w2", dtype=tf.float32)
    b2 = tf.Variable(tf.zeros([h2_size]),name="g_b2", dtype=tf.float32)
    h2 = tf.nn.relu(tf.matmul(h1,w2)+b2)
        
    w3 = tf.Variable(tf.truncated_normal([h2_size, img_size],stddev=0.1),name="g_w3", dtype=tf.float32)
    b3 = tf.Variable(tf.zeros([img_size]),name="g_b3", dtype=tf.float32)
    h3 = tf.matmul(h2,w3)+b3
    
    g_params = [w1,b1,w2,b2,w3,b3]
    x_generate = tf.nn.tanh(h3)
       
    return x_generate, g_params
    # ---------------------------------------

In [4]:
# 디스크리미네이터 (D)

    # ----------- 코드 작성부분 ----------------

def discriminator(x_data, x_generated, keep_prob):
    x_in = tf.concat([x_data, x_generated],0)
            
    w1 = tf.Variable(tf.truncated_normal([img_size,h2_size],stddev=0.1),name='d_w1', dtype= tf.float32)
    b1 = tf.Variable(tf.zeros([h2_size]),name="d_b1", dtype=tf.float32)
    h1 = tf.nn.dropout(tf.nn.relu(tf.matmul(x_in,w1)+b1),keep_prob)
        
    w2 = tf.Variable(tf.truncated_normal([h2_size,h1_size],stddev=0.1),name='d_w2', dtype= tf.float32)
    b2 = tf.Variable(tf.zeros([h1_size]),name="d_b2", dtype=tf.float32)
    h2 = tf.nn.dropout(tf.nn.relu(tf.matmul(h1,w2)+b2),keep_prob)
        
    w3 = tf.Variable(tf.truncated_normal([h1_size,1],stddev=0.1),name='d_w3', dtype= tf.float32)
    b3 = tf.Variable(tf.zeros([1]),name="d_b3", dtype=tf.float32)
    h3 = tf.matmul(h2,w3)+b3
        
    y_data = tf.nn.sigmoid(tf.slice(h3,[0,0],[batch_size,-1],name=None))
    y_generated = tf.nn.sigmoid(tf.slice(h3,[batch_size,0],[-1,-1],name=None))
        
    d_params = [w1,b1,w2,b2,w3,b3]
        
    return y_data, y_generated, d_params
    # ---------------------------------------
    
    

In [5]:
def show_result(batch_res, fname, grid_size=(8, 8), grid_pad=5):
    batch_res = 0.5 * batch_res.reshape((batch_res.shape[0], img_height, img_width)) + 0.5
    img_h, img_w = batch_res.shape[1], batch_res.shape[2]
    grid_h = img_h * grid_size[0] + grid_pad * (grid_size[0] - 1)
    grid_w = img_w * grid_size[1] + grid_pad * (grid_size[1] - 1)
    img_grid = np.zeros((grid_h, grid_w), dtype=np.uint8)
    for i, res in enumerate(batch_res):
        if i >= grid_size[0] * grid_size[1]:
            break
        img = (res) * 255.
        img = img.astype(np.uint8) 
        row = (i // grid_size[0]) * (img_h + grid_pad)
        col = (i % grid_size[1]) * (img_w + grid_pad)
        img_grid[row:row + img_h, col:col + img_w] = img
    imsave(fname, img_grid)

In [ ]:
def train():
    
    global_step = tf.Variable(0, name="global_step", trainable=False)
    
    
    # ----------- 코드 작성부분 ----------------
    
    train_x, train_y = load_mnist("fashion_mnist",kind='train')
    size = train_x.shape[0]
    x_data = tf.placeholder(tf.float32, [batch_size,img_size],name="x_data")
    z=tf.placeholder(tf.float32, [batch_size,z_size],name="z")
    keep_prob = tf.placeholder(tf.float32,name="keep_prob")
    
    x_generated, g_params = generator(z)
    
    y_data, y_generated, d_params = discriminator(x_data, x_generated, keep_prob)

    # ---------------------------------------

    d_loss = -(tf.log(y_data)+tf.log(1-y_generated))
    g_loss = -tf.log(y_generated)
    
    optimizer = tf.train.AdamOptimizer(0.0001)
    
    d_trainer = optimizer.minimize(d_loss,var_list=d_params)
    g_trainer = optimizer.minimize(g_loss,var_list=g_params)
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
    sess = tf.Session()
            
    sess.run(init)
    
    if to_restore:
        chkpt_fname = tf.train.latest_checkpoint(output_path)
        saver.restore(sess, chkpt_fname)
    else:
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.mkdir(output_path)

    z_sample_val = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)

    
    for i in range(sess.run(global_step), max_epoch):
        for j in range(21870 // batch_size):
            print("epoch:%s, iter:%s" % (i, j))

            batch_end = j * batch_size + batch_size
            if batch_end >= size:
                batch_end = size - 1
            x_value = train_x[ j * batch_size : batch_end ]
            x_value = x_value / 255.
            x_value = 2 * x_value - 1

            z_value = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
            sess.run(d_trainer,
                     feed_dict={x_data: x_value, z: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
            if j % 1 == 0:
                sess.run(g_trainer,
                         feed_dict={x_data: x_value, z: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
        x_gen_val = sess.run(x_generated, feed_dict={z: z_sample_val})
        show_result(x_gen_val, os.path.join(output_path, "sample%s.jpg" % i))
        sess.run(tf.assign(global_step, i + 1))
    
    saver.save(sess, 'output2/GDG_CODLAB_STUDY.ckpt')

In [ ]:
train()

INFO:tensorflow:Restoring parameters from output2\GDG_CODLAB_STUDY.ckpt
epoch:50, iter:0
epoch:50, iter:1
epoch:50, iter:2
epoch:50, iter:3
epoch:50, iter:4
epoch:50, iter:5
epoch:50, iter:6
epoch:50, iter:7
epoch:50, iter:8
epoch:50, iter:9
epoch:50, iter:10
epoch:50, iter:11
epoch:50, iter:12
epoch:50, iter:13
epoch:50, iter:14
epoch:50, iter:15
epoch:50, iter:16
epoch:50, iter:17
epoch:50, iter:18
epoch:50, iter:19
epoch:50, iter:20
epoch:50, iter:21
epoch:50, iter:22
epoch:50, iter:23
epoch:50, iter:24
epoch:50, iter:25
epoch:50, iter:26
epoch:50, iter:27
epoch:50, iter:28
epoch:50, iter:29
epoch:50, iter:30
epoch:50, iter:31
epoch:50, iter:32
epoch:50, iter:33
epoch:50, iter:34
epoch:50, iter:35
epoch:50, iter:36
epoch:50, iter:37
epoch:50, iter:38
epoch:50, iter:39
epoch:50, iter:40
epoch:50, iter:41
epoch:50, iter:42
epoch:50, iter:43
epoch:50, iter:44
epoch:50, iter:45
epoch:50, iter:46
epoch:50, iter:47
epoch:50, iter:48
epoch:50, iter:49
epoch:50, iter:50
epoch:50, iter:51
ep

epoch:55, iter:34
epoch:55, iter:35
epoch:55, iter:36
epoch:55, iter:37
epoch:55, iter:38
epoch:55, iter:39
epoch:55, iter:40
epoch:55, iter:41
epoch:55, iter:42
epoch:55, iter:43
epoch:55, iter:44
epoch:55, iter:45
epoch:55, iter:46
epoch:55, iter:47
epoch:55, iter:48
epoch:55, iter:49
epoch:55, iter:50
epoch:55, iter:51
epoch:55, iter:52
epoch:55, iter:53
epoch:55, iter:54
epoch:55, iter:55
epoch:55, iter:56
epoch:55, iter:57
epoch:55, iter:58
epoch:55, iter:59
epoch:55, iter:60
epoch:55, iter:61
epoch:55, iter:62
epoch:55, iter:63
epoch:55, iter:64
epoch:55, iter:65
epoch:55, iter:66
epoch:55, iter:67
epoch:55, iter:68
epoch:55, iter:69
epoch:55, iter:70
epoch:55, iter:71
epoch:55, iter:72
epoch:55, iter:73
epoch:55, iter:74
epoch:55, iter:75
epoch:55, iter:76
epoch:55, iter:77
epoch:55, iter:78
epoch:55, iter:79
epoch:55, iter:80
epoch:55, iter:81
epoch:55, iter:82
epoch:55, iter:83
epoch:55, iter:84
epoch:56, iter:0
epoch:56, iter:1
epoch:56, iter:2
epoch:56, iter:3
epoch:56, iter

epoch:60, iter:67
epoch:60, iter:68
epoch:60, iter:69
epoch:60, iter:70
epoch:60, iter:71
epoch:60, iter:72
epoch:60, iter:73
epoch:60, iter:74
epoch:60, iter:75
epoch:60, iter:76
epoch:60, iter:77
epoch:60, iter:78
epoch:60, iter:79
epoch:60, iter:80
epoch:60, iter:81
epoch:60, iter:82
epoch:60, iter:83
epoch:60, iter:84
epoch:61, iter:0
epoch:61, iter:1
epoch:61, iter:2
epoch:61, iter:3
epoch:61, iter:4
epoch:61, iter:5
epoch:61, iter:6
epoch:61, iter:7
epoch:61, iter:8
epoch:61, iter:9
epoch:61, iter:10
epoch:61, iter:11
epoch:61, iter:12
epoch:61, iter:13
epoch:61, iter:14
epoch:61, iter:15
epoch:61, iter:16
epoch:61, iter:17
epoch:61, iter:18
epoch:61, iter:19
epoch:61, iter:20
epoch:61, iter:21
epoch:61, iter:22
epoch:61, iter:23
epoch:61, iter:24
epoch:61, iter:25
epoch:61, iter:26
epoch:61, iter:27
epoch:61, iter:28
epoch:61, iter:29
epoch:61, iter:30
epoch:61, iter:31
epoch:61, iter:32
epoch:61, iter:33
epoch:61, iter:34
epoch:61, iter:35
epoch:61, iter:36
epoch:61, iter:37
ep

epoch:66, iter:18
epoch:66, iter:19
epoch:66, iter:20
epoch:66, iter:21
epoch:66, iter:22
epoch:66, iter:23
epoch:66, iter:24
epoch:66, iter:25
epoch:66, iter:26
epoch:66, iter:27
epoch:66, iter:28
epoch:66, iter:29
epoch:66, iter:30
epoch:66, iter:31
epoch:66, iter:32
epoch:66, iter:33
epoch:66, iter:34
epoch:66, iter:35
epoch:66, iter:36
epoch:66, iter:37
epoch:66, iter:38
epoch:66, iter:39
epoch:66, iter:40
epoch:66, iter:41
epoch:66, iter:42
epoch:66, iter:43
epoch:66, iter:44
epoch:66, iter:45
epoch:66, iter:46
epoch:66, iter:47
epoch:66, iter:48
epoch:66, iter:49
epoch:66, iter:50
epoch:66, iter:51
epoch:66, iter:52
epoch:66, iter:53
epoch:66, iter:54
epoch:66, iter:55
epoch:66, iter:56
epoch:66, iter:57
epoch:66, iter:58
epoch:66, iter:59
epoch:66, iter:60
epoch:66, iter:61
epoch:66, iter:62
epoch:66, iter:63
epoch:66, iter:64
epoch:66, iter:65
epoch:66, iter:66
epoch:66, iter:67
epoch:66, iter:68
epoch:66, iter:69
epoch:66, iter:70
epoch:66, iter:71
epoch:66, iter:72
epoch:66, 

epoch:71, iter:52
epoch:71, iter:53
epoch:71, iter:54
epoch:71, iter:55
epoch:71, iter:56
epoch:71, iter:57
epoch:71, iter:58
epoch:71, iter:59
epoch:71, iter:60
epoch:71, iter:61
epoch:71, iter:62
epoch:71, iter:63
epoch:71, iter:64
epoch:71, iter:65
epoch:71, iter:66
epoch:71, iter:67
epoch:71, iter:68
epoch:71, iter:69
epoch:71, iter:70
epoch:71, iter:71
epoch:71, iter:72
epoch:71, iter:73
epoch:71, iter:74
epoch:71, iter:75
epoch:71, iter:76
epoch:71, iter:77
epoch:71, iter:78
epoch:71, iter:79
epoch:71, iter:80
epoch:71, iter:81
epoch:71, iter:82
epoch:71, iter:83
epoch:71, iter:84
epoch:72, iter:0
epoch:72, iter:1
epoch:72, iter:2
epoch:72, iter:3
epoch:72, iter:4
epoch:72, iter:5
epoch:72, iter:6
epoch:72, iter:7
epoch:72, iter:8
epoch:72, iter:9
epoch:72, iter:10
epoch:72, iter:11
epoch:72, iter:12
epoch:72, iter:13
epoch:72, iter:14
epoch:72, iter:15
epoch:72, iter:16
epoch:72, iter:17
epoch:72, iter:18
epoch:72, iter:19
epoch:72, iter:20
epoch:72, iter:21
epoch:72, iter:22
ep

...

epoch:206, iter:33
epoch:206, iter:34
epoch:206, iter:35
epoch:206, iter:36
epoch:206, iter:37
epoch:206, iter:38
epoch:206, iter:39
epoch:206, iter:40
epoch:206, iter:41
epoch:206, iter:42
epoch:206, iter:43
epoch:206, iter:44
epoch:206, iter:45
epoch:206, iter:46
epoch:206, iter:47
epoch:206, iter:48
epoch:206, iter:49
epoch:206, iter:50
epoch:206, iter:51
epoch:206, iter:52
epoch:206, iter:53
epoch:206, iter:54
epoch:206, iter:55
epoch:206, iter:56
epoch:206, iter:57
epoch:206, iter:58
epoch:206, iter:59
epoch:206, iter:60
epoch:206, iter:61
epoch:206, iter:62
epoch:206, iter:63
epoch:206, iter:64
epoch:206, iter:65
epoch:206, iter:66
epoch:206, iter:67
epoch:206, iter:68
epoch:206, iter:69
epoch:206, iter:70
epoch:206, iter:71
epoch:206, iter:72
epoch:206, iter:73
epoch:206, iter:74
epoch:206, iter:75
epoch:206, iter:76
epoch:206, iter:77
epoch:206, iter:78
epoch:206, iter:79
epoch:206, iter:80
epoch:206, iter:81
epoch:206, iter:82
epoch:206, iter:83
epoch:206, iter:84
epoch:207, i

epoch:211, iter:42
epoch:211, iter:43
epoch:211, iter:44
epoch:211, iter:45
epoch:211, iter:46
epoch:211, iter:47
epoch:211, iter:48
epoch:211, iter:49
epoch:211, iter:50
epoch:211, iter:51
epoch:211, iter:52
epoch:211, iter:53
epoch:211, iter:54
epoch:211, iter:55
epoch:211, iter:56
epoch:211, iter:57
epoch:211, iter:58
epoch:211, iter:59
epoch:211, iter:60
epoch:211, iter:61
epoch:211, iter:62
epoch:211, iter:63
epoch:211, iter:64
epoch:211, iter:65
epoch:211, iter:66
epoch:211, iter:67
epoch:211, iter:68
epoch:211, iter:69
epoch:211, iter:70
epoch:211, iter:71
epoch:211, iter:72
epoch:211, iter:73
epoch:211, iter:74
epoch:211, iter:75
epoch:211, iter:76
epoch:211, iter:77
epoch:211, iter:78
epoch:211, iter:79
epoch:211, iter:80
epoch:211, iter:81
epoch:211, iter:82
epoch:211, iter:83
epoch:211, iter:84
epoch:212, iter:0
epoch:212, iter:1
epoch:212, iter:2
epoch:212, iter:3
epoch:212, iter:4
epoch:212, iter:5
epoch:212, iter:6
epoch:212, iter:7
epoch:212, iter:8
epoch:212, iter:9
epo

epoch:216, iter:51
epoch:216, iter:52
epoch:216, iter:53
epoch:216, iter:54
epoch:216, iter:55
epoch:216, iter:56
epoch:216, iter:57
epoch:216, iter:58
epoch:216, iter:59
epoch:216, iter:60
epoch:216, iter:61
epoch:216, iter:62
epoch:216, iter:63
epoch:216, iter:64
epoch:216, iter:65
epoch:216, iter:66
epoch:216, iter:67
epoch:216, iter:68
epoch:216, iter:69
epoch:216, iter:70
epoch:216, iter:71
epoch:216, iter:72
epoch:216, iter:73
epoch:216, iter:74
epoch:216, iter:75
epoch:216, iter:76
epoch:216, iter:77
epoch:216, iter:78
epoch:216, iter:79
epoch:216, iter:80
epoch:216, iter:81
epoch:216, iter:82
epoch:216, iter:83
epoch:216, iter:84
epoch:217, iter:0
epoch:217, iter:1
epoch:217, iter:2
epoch:217, iter:3
epoch:217, iter:4
epoch:217, iter:5
epoch:217, iter:6
epoch:217, iter:7
epoch:217, iter:8
epoch:217, iter:9
epoch:217, iter:10
epoch:217, iter:11
epoch:217, iter:12
epoch:217, iter:13
epoch:217, iter:14
epoch:217, iter:15
epoch:217, iter:16
epoch:217, iter:17
epoch:217, iter:18
epo

epoch:221, iter:60
epoch:221, iter:61
epoch:221, iter:62
epoch:221, iter:63
epoch:221, iter:64
epoch:221, iter:65
epoch:221, iter:66
epoch:221, iter:67
epoch:221, iter:68
epoch:221, iter:69
epoch:221, iter:70
epoch:221, iter:71
epoch:221, iter:72
epoch:221, iter:73
epoch:221, iter:74
epoch:221, iter:75
epoch:221, iter:76
epoch:221, iter:77
epoch:221, iter:78
epoch:221, iter:79
epoch:221, iter:80
epoch:221, iter:81
epoch:221, iter:82
epoch:221, iter:83
epoch:221, iter:84
epoch:222, iter:0
epoch:222, iter:1
epoch:222, iter:2
epoch:222, iter:3
epoch:222, iter:4
epoch:222, iter:5
epoch:222, iter:6
epoch:222, iter:7
epoch:222, iter:8
epoch:222, iter:9
epoch:222, iter:10
epoch:222, iter:11
epoch:222, iter:12
epoch:222, iter:13
epoch:222, iter:14
epoch:222, iter:15
epoch:222, iter:16
epoch:222, iter:17
epoch:222, iter:18
epoch:222, iter:19
epoch:222, iter:20
epoch:222, iter:21
epoch:222, iter:22
epoch:222, iter:23
epoch:222, iter:24
epoch:222, iter:25
epoch:222, iter:26
epoch:222, iter:27
epo

epoch:226, iter:72
epoch:226, iter:73
epoch:226, iter:74
epoch:226, iter:75
epoch:226, iter:76
epoch:226, iter:77
epoch:226, iter:78
epoch:226, iter:79
epoch:226, iter:80
epoch:226, iter:81
epoch:226, iter:82
epoch:226, iter:83
epoch:226, iter:84
epoch:227, iter:0
epoch:227, iter:1
epoch:227, iter:2
epoch:227, iter:3
epoch:227, iter:4
epoch:227, iter:5
epoch:227, iter:6
epoch:227, iter:7
epoch:227, iter:8
epoch:227, iter:9
epoch:227, iter:10
epoch:227, iter:11
epoch:227, iter:12
epoch:227, iter:13
epoch:227, iter:14
epoch:227, iter:15
epoch:227, iter:16
epoch:227, iter:17
epoch:227, iter:18
epoch:227, iter:19
epoch:227, iter:20
epoch:227, iter:21
epoch:227, iter:22
epoch:227, iter:23
epoch:227, iter:24
epoch:227, iter:25
epoch:227, iter:26
epoch:227, iter:27
epoch:227, iter:28
epoch:227, iter:29
epoch:227, iter:30
epoch:227, iter:31
epoch:227, iter:32
epoch:227, iter:33
epoch:227, iter:34
epoch:227, iter:35
epoch:227, iter:36
epoch:227, iter:37
epoch:227, iter:38
epoch:227, iter:39
epo